In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

# ==========================================
# 1. データベースの準備 (DB作成・テーブル作成)
# ==========================================
db_name = 'google_repos.db'
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# テーブルが存在する場合は一度削除して作り直す（リセット用）
cur.execute("DROP TABLE IF EXISTS repositories")

# テーブル作成: リポジトリ名(name), 言語(language), スター数(stars)
# スター数は "1.2k" などの表記があるため TEXT 型として扱いますが、数値変換しても構いません
sql_create = 'CREATE TABLE repositories (name TEXT, language TEXT, stars TEXT);'
cur.execute(sql_create)

# ==========================================
# 2. スクレイピングの実行
# ==========================================
target_url = 'https://github.com/google?tab=repositories'

print(f"Scraping: {target_url}")

try:
    # GitHubはUser-Agentがないと拒否されることがあるためヘッダーを設定
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    res = requests.get(target_url, headers=headers, timeout=10)
    
    # ※重要: リクエスト間の待機（課題要件）
    time.sleep(0.5)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # リポジトリのリスト要素を取得 (liタグの特定クラスを探す)
        # GitHubのHTML構造に依存するため、クラス名は変更される可能性がありますが、
        # 一般的な "public source" クラスを持つ li を対象にします。
        repo_list = soup.find_all('li', class_='col-12 d-flex width-full py-4 border-bottom color-border-muted public source')
        
        print(f"Found {len(repo_list)} repositories on the first page.\n")

        data_to_insert = []

        for repo in repo_list:
            # --- リポジトリ名の取得 ---
            # itemprop="name codeRepository" を持つ aタグ
            name_tag = repo.find('a', itemprop='name codeRepository')
            name = name_tag.get_text(strip=True) if name_tag else "Unknown"
            
            # --- 主要言語の取得 ---
            # itemprop="programmingLanguage" を持つ spanタグ
            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.get_text(strip=True) if lang_tag else "None"
            
            # --- スター数の取得 ---
            # hrefが "/stargazers" で終わる aタグを探す
            star_tag = repo.find('a', href=lambda href: href and href.endswith('/stargazers'))
            stars = star_tag.get_text(strip=True) if star_tag else "0"
            
            # データのタプルを作成
            data_to_insert.append((name, language, stars))

        # ==========================================
        # 3. データの保存 (INSERT)
        # ==========================================
        sql_insert = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);"
        cur.executemany(sql_insert, data_to_insert)
        conn.commit()
        print("データの保存が完了しました。\n")

    else:
        print(f"Failed to retrieve page. Status code: {res.status_code}")

except Exception as e:
    print(f"エラーが発生しました: {e}")

# ==========================================
# 4. データの確認 (SELECT)
# ==========================================
print("=== 保存されたデータの確認 (SELECT実行) ===")
try:
    cur.execute("SELECT * FROM repositories")
    rows = cur.fetchall()
    
    # ヘッダー出力
    print(f"{'Name':<40} | {'Language':<15} | {'Stars':<10}")
    print("-" * 70)
    
    for row in rows:
        r_name, r_lang, r_stars = row
        print(f"{r_name:<40} | {r_lang:<15} | {r_stars:<10}")

except sqlite3.Error as e:
    print('SELECTエラー:', e)

finally:
    conn.close()